---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 
          'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 
          'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 
          'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin',
          'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 
          'MS': 'Mississippi','PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
          'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 
          'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 
          'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 
          'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 
          'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 
          'ND': 'North Dakota','VA': 'Virginia'}

In [62]:
def get_list_of_university_towns():
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
        
    university_towns = pd.read_csv("university_towns.txt", 
                                   sep = '\n', 
                                   header = None, 
                                   names = ["RegionName"])
        
    university_towns["State"] = (university_towns.iloc[:,0]
                                 .str.extract("(.*)\[edit\]", 
                                              expand = False)
                                 .ffill())
    #State and city are both in one column in the data
    #extract the state name into a new column, then forward filled the state name until 
    #the next state was reached, and so on until the whole column is populated.
    
    university_towns = university_towns.loc[~university_towns["RegionName"]
                                            .str.contains("\[edit\]")] 
    #remove duplicate data since new column with state name was created
    
    university_towns["RegionName"].replace(" \((.*)","",
                                           inplace = True,
                                           regex = True) 
    #finds " (" and removes the expression and anything following.
                                           
    university_towns = university_towns.reindex(columns = ["State","RegionName"])
    #switch the order of the columns    
    
    university_towns.set_index(["State","RegionName"],
                               inplace = True,
                               drop = False)
    #create multi index
    
    return university_towns

get_list_of_university_towns()

State     RegionName
State     RegionName                             
Alabama   Auburn           Alabama         Auburn
          Florence         Alabama       Florence
          Jacksonville     Alabama   Jacksonville
          Livingston       Alabama     Livingston
          Montevallo       Alabama     Montevallo
...                            ...            ...
Wisconsin River Falls    Wisconsin    River Falls
          Stevens Point  Wisconsin  Stevens Point
          Waukesha       Wisconsin       Waukesha
          Whitewater     Wisconsin     Whitewater
Wyoming   Laramie          Wyoming        Laramie

[517 rows x 2 columns]

In [50]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
        
    #Created two new columns, one for the change in GDP quarter over quarter. 
    #The other column contains indicators for whether there was a two quarter decline in GDP
    #Added an indicator variable for No_recession so only the beginning of the recession is recorded
    
    gdp = pd.read_excel("gdplev.xls", 
                        skiprows = 8, 
                        header = None)
    
    gdp = (gdp.iloc[:,[4,6]]
           .rename(columns = {4: "Quarter", 
                              6: "GDP 2009 Chained Value"})) 

    gdp = gdp.loc[gdp["Quarter"] >= "2000q1"]
    #remove data prior to Q1 2000    
        
    gdp["Change in GDP"] = 0
    gdp["Recession Begin Indicator"] = False
    
    recession = False
    for i in range(len(gdp)-1):
        gdp.iloc[i+1,2] = gdp.iloc[i+1,1] - gdp.iloc[i,1]
        if gdp.iloc[i+1,2] < 0 and gdp.iloc[i,2] < 0 and recession == False:
            gdp.iloc[i,3] = True
            recession = True
        else:
            pass
    #calculate the change in GDP, if there are two consecutive quarters of negative growth,
    #mark the beginning of the recession

    recession_start = (gdp.loc[gdp["Recession Begin Indicator"] == True,"Quarter"]
                       .iloc[0])
    
    return recession_start

get_recession_start()

'2008q3'

In [55]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    #Uses same code from above to create the recession indicator column.
    #created a new column for date time values using the "Quarter" column to compare the two quarters of 
    #consecutive growth to the beginning of the recession. 
    
    gdp = pd.read_excel("gdplev.xls", 
                        skiprows = 8, 
                        header = None)
    
    gdp = gdp.iloc[:,[4,6]].rename(columns = {4: "Quarter", 
                                              6: "GDP 2009 Chained Value"})
    
    gdp = gdp.loc[gdp["Quarter"] >= "2000q1"]
    gdp["Change in GDP"] = 0
    gdp["Recession Begin Indicator"] = False
    gdp["Recession End Indicator"] = False
    gdp["Datetime"] = pd.to_datetime(gdp["Quarter"])
    
    recession = False
    for i in range(len(gdp)-1):
        gdp.iloc[i+1,2] = gdp.iloc[i+1,1] - gdp.iloc[i,1]
        if gdp.iloc[i+1,2] < 0 and gdp.iloc[i,2] < 0 and recession == False:
            gdp.iloc[i,3] = True
            recession = True
        else:
            pass
    
    #new code starts here        
    recession_start = (gdp.loc[gdp["Recession Begin Indicator"] == True,"Datetime"]
                       .iloc[0])
    
    recovery = False
    for i in range(len(gdp)-2):
        if (
        recovery == False and 
        gdp.iloc[i+1,2] > 0 and 
        gdp.iloc[i+2,2] > 0 and 
        gdp.iloc[i+1,5] > recession_start):
            
            gdp.iloc[i+2,4] = True
            recovery = True
        #check for two consecutive quarters of growth that occur after the recession has started
        #and mark this as the beginning of the recovery  
        
        else:
            pass
            
    recession_end = (gdp.loc[gdp["Recession End Indicator"] == True, "Quarter"]
                     .iloc[0])

    return recession_end

get_recession_end()

'2009q4'

In [58]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    #uses the return values of get_recession_start() and get_recession_bottom() to create a new dataframe with 
    #only data between those two points in time, i.e. the recession period. Returns the quarter with the lowest
    #GDP during the recession.
    
    gdp = pd.read_excel("gdplev.xls", 
                        skiprows = 8, 
                        header = None)
    
    gdp = gdp.iloc[:,[4,6]].rename(columns = {4: "Quarter", 
                                              6: "GDP 2009 Chained Value"})
    
    gdp = gdp.loc[gdp["Quarter"] >= "2000q1"]
    
    gdp.set_index(keys = gdp["Quarter"], 
                  inplace = True)
        
    recession = gdp.loc[get_recession_start() : get_recession_end()]
    #use previously defined function return values as arguments for the dataframe slice
    
    recession_bottom = recession["GDP 2009 Chained Value"].idxmin()
        
    return recession_bottom

get_recession_bottom()


'2009q2'

In [72]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
        
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
        
    housing_data = pd.read_csv("City_Zhvi_AllHomes.csv")
    
    housing_data["State"].replace(states, 
                                  inplace = True)
    
    housing_data.set_index(["State","RegionName"],
                           inplace = True)
       
    mean_qtr_housing_data = (housing_data.loc[:,"2000-01":"2016-08"]
                             .groupby(pd.PeriodIndex(housing_data.loc[:,"2000-01":"2016-08"],
                                                                                  freq = 'Q'), 
                                                                                  axis = 1)
                             .mean()
                             .rename(columns = lambda c: str(c).lower())) 
    #create a quarter-wise index and groupby quarter, return the 
    #mean value for each quarter in the specified period
        
    return mean_qtr_housing_data

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [76]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    #pulling everything together. Merge the university_towns df onto the mean_qtr_housing df and create an indicator column
    #for whether the state/regionname is a university town or non-university town.
    #use the college town indicator to create two separate dataframes, one containing only university towns and the other 
    #containing only non-university towns
    #use the return values of recession_start and recession bottom to create two series (one of university town and one of non-university town)
    #of price ratios, where price ratio = (quarter before recession)/(recession bottom) 
    #run a t test for difference of means to see if we can reject the null hypothesis at a significance level of 0.01
    #return different if the p-value of the ttest <0.01, return the p value, and return the subset of data with the lower mean price ratio.
    #a lower ratio indicates a smaller price difference from the beginning to the bottom of the recession
    
    mean_qtr_housing_data = convert_housing_data_to_quarters()

    mean_qtr1 = (mean_qtr_housing_data
                 .merge(get_list_of_university_towns(), 
                        how = "left", 
                        left_index = True, 
                        right_index = True))
    
    mean_qtr1["College Town Indicator"] = False
    for i in range(len(mean_qtr1)):
        if pd.isnull(mean_qtr1.iloc[i,68]) == False:
            mean_qtr1.iloc[i,69] = True
            
    mean_qtr1.drop(["State","RegionName"],
                   axis = 1, 
                   inplace = True)
    
    mean_qtr1_university_town = (mean_qtr1
                                 .loc[mean_qtr1["College Town Indicator"] == True,
                                      :])
    
    mean_qtr1_non_university_town = (mean_qtr1
                                     .loc[mean_qtr1["College Town Indicator"] == False,
                                          :])
    
    university_price_ratio = (mean_qtr1_university_town[get_recession_start()]
                              /mean_qtr1_university_town[get_recession_bottom()])
    
    non_university_price_ratio = (mean_qtr1_non_university_town[get_recession_start()]
                                  /mean_qtr1_non_university_town[get_recession_bottom()])
    
    ttest = ttest_ind(university_price_ratio,
                      non_university_price_ratio, 
                      nan_policy = "omit")
    
    #I think this part should be a separate function
    
    different, p, better = str, float, str    
    
    p = ttest[1]
    
    if p < 0.01:
        different = True
    else:
        different = False

    if university_price_ratio.mean() < non_university_price_ratio.mean():
        better = "university town"
    else:
        better = "non-university town"
        
    answer = (different, p, better)
           
    return answer

run_ttest()

(True, 0.005496427353633026, 'university town')